In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os, pickle, shutil, random, PIL
from PIL import Image

import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader,random_split,Dataset, ConcatDataset ,SubsetRandomSampler 
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.transforms import v2

import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold

from training_utils import *
from cam_loss_training_utils import *
from classification_models import *
from focal_loss_with_smoothing import *

from torchcam.methods import SmoothGradCAMpp, LayerCAM, GradCAM,CAM
from torchcam.utils import overlay_mask
from torchvision.transforms.functional import to_pil_image, resize
from torch.nn.functional import softmax, interpolate
from copy import deepcopy
from sklearn.metrics import accuracy_score

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

DFNAME = 'MB_34_Layer_1_L2_freezeb5'
device = torch.device('cuda:0')
criterion1 = FocalLossWithSmoothing(num_classes = 2, gamma=2, lb_smooth = 0.1)
criterion2 =  nn.MSELoss()
# criterion1 = nn.CrossEntropyLoss()

modelname = 'MB_34_Layer_1_L2_freezeb5'
n_epochs = 50
batch_size = 8

In [3]:
train_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/train_original/imgs/'
test_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/test/imgs/'
val_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/val/imgs/'

In [4]:
size = (1024,768)
train_set_whole = ImageFolder(train_dir,transform = transforms.Compose([
    # v2.Grayscale(1),
    v2.RandomVerticalFlip(0.5),
    v2.RandomHorizontalFlip(0.5),
    v2.RandomRotation(15),
    v2.RandomAutocontrast(0.5),
    # v2.ColorJitter(brightness=0.4, contrast=0.2, saturation=0.3, hue=0.1),
    # v2.RandomChannelPermutation(),
    v2.RandomAdjustSharpness(2,0.5),
    # v2.RandomAutocontrast(0.5),
    v2.Resize(size),
    # transforms.GaussianBlur(kernel_size=3),
    # transforms.RandomRotation(30),
    v2.ToTensor(),
]))

val_set = ImageFolder(val_dir,transform = transforms.Compose([
    # v2.Grayscale(1),
    v2.Resize(size),
    v2.ToTensor(),
]))

test_set = ImageFolder(test_dir,transform = transforms.Compose([
    # v2.Grayscale(1),
    v2.Resize(size),
    v2.ToTensor(),
]))

In [5]:
train_loader = DataLoader(train_set_whole, batch_size=batch_size, shuffle=True, num_workers = 4)
valid_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers = 4)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers = 4)
test_loader_2 = DataLoader(test_set, batch_size=1, shuffle=False)

In [6]:
new_model =custom_rn(2).to(device)

checkpoint = torch.load('./checkpoint/'+modelname+'model.pth.tar',map_location=torch.device('cpu'))
new_model.load_state_dict(checkpoint['model'])

_, _, y, y_pred = best_test(new_model,test_loader_2,criterion1,optim,device,1)

100%|███████████████████████████████████████████████████████████████████████████████████| 67/67 [00:03<00:00, 17.53it/s]

Epoch: [2]  loss: [0.19] Test Accuracy [86.57] 


In [7]:
test_benign_img_names = os.listdir("/home/aminul/CVL/cs791_project/malignant_vs_benign/test/imgs/0_benign/")
test_malignant_img_names = os.listdir("/home/aminul/CVL/cs791_project/malignant_vs_benign/test/imgs/1_malignant/")

train_benign_img_names = os.listdir("/home/aminul/CVL/cs791_project/malignant_vs_benign/train_original/imgs/0_aug_benign/")
train_malignant_img_names = os.listdir("/home/aminul/CVL/cs791_project/malignant_vs_benign/train_original/imgs/1_aug_malignant/")

val_benign_img_names = os.listdir("/home/aminul/CVL/cs791_project/malignant_vs_benign/val/imgs/0_benign/")
val_malignant_img_names = os.listdir("/home/aminul/CVL/cs791_project/malignant_vs_benign/val/imgs/1_malignant/")


all_imgs_names_val = val_benign_img_names+val_malignant_img_names

all_imgs_names_test = test_benign_img_names+test_malignant_img_names

all_imgs_names_train = train_benign_img_names+train_malignant_img_names

In [8]:
test_labels,train_labels, val_labels = [],[],[]

for i in range(len(test_benign_img_names)):
    test_labels.append(0)
    
for i in range(len(test_malignant_img_names)):
    test_labels.append(1)
    
    
for i in range(len(val_benign_img_names)):
    val_labels.append(0)
    
for i in range(len(val_malignant_img_names)):
    val_labels.append(1)
    

for i in range(len(train_benign_img_names)):
    train_labels.append(0)
    
for i in range(len(train_malignant_img_names)):
    train_labels.append(1)

In [12]:
# img_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/test/combined_test_imgs/'
# mask_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/test/combined_masks/'

# img_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/train_original/combined_imgs/'
# mask_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/train_original/combined_masks/'

img_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/val/combined_imgs/'
mask_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/train_original/combined_masks/'

img_filenames = os.listdir(img_dir)
mask_filenames = os.listdir(mask_dir)

In [13]:
test_transform = v2.Compose([
    v2.Resize((1024,768)),
    v2.ToTensor(),
    ])

In [14]:
def sml_func(percentage):

        # if percentage < 0.5:
        #     return 3 ## small
        # elif percentage >= 0.5 and percentage <3:
        #     return 4
        # elif percentage >= 3:
        #     return 5
        
        
        if percentage < 0.5:
            return 3 ## small
        elif percentage >= 0.5 and percentage <3:
            return 4
        elif percentage >= 0.5:
            return 5
            

In [15]:
i = 10
img_name, mask_values, percentage,sml_label = [],[],[],[]

y_pred, y_true = [],[]


for i in range(len(img_filenames)):

    img = Image.open(img_dir + img_filenames[i])
    ori_img = img.convert("RGB") 
    
    idx = all_imgs_names_val.index(img_filenames[i]) ### Change based on train or test
    y_true.append(val_labels[idx]) ### Change based on train or test
    
    mask = Image.open(mask_dir + img_filenames[i])
    mask = mask.convert('L')
    threshold = 100
    mask = mask.point(lambda p: p > threshold and 255)
    mask_np = np.array(mask)
    counted_values = np.count_nonzero(mask_np == 255)
    mask_values.append(counted_values)
    per = (counted_values / (mask_np.shape[0] * mask_np.shape[1]))*100
    percentage.append(per)
    sml_label.append(sml_func(per))
    
    
    img1 = test_transform(copy.deepcopy(ori_img))
    img1 = img1.to(device)
    
    new_model.eval()
    with torch.no_grad():
        outputs,_,_ = new_model(img1.unsqueeze(0))
        _,predicted = torch.max(outputs.data,1)

        y_pred.append(predicted.cpu().numpy())
    

In [17]:
for i in range(len(y_pred)):
    
    if y_pred[i] != y_true[i]:
        print(y_pred[i], y_true[i],sml_label[i],percentage[i])

[0] 1 3 0.00014133516503707223
[0] 1 5 16.36893277102736
[0] 1 3 0.0003406729426193207
[0] 1 3 0.4520248611337578


In [16]:
accuracy_score(y_true,y_pred)

0.8461538461538461

In [18]:
sml_label.count(3),sml_label.count(4),sml_label.count(5)

(19, 4, 3)